# Example: field plan from Sudoku design

First, import all required modules.

In [ ]:
using SudokuPlantDesign
using DataFrames
using XLSX
using PyPlot

Note that one can make a field plan for any design featured in `SudokuPlantDesign.jl`, however in this example, we will be closely resembling the augmented design with replicated checks and unreplicated entries, featured [here](https://github.com/janattig/SudokuPlantDesign.jl/blob/main/examples/sudoku_augmented.ipynb).

## Generating an (optimized) Sudoku configuration

To create a field plan for a design, one first has to go through the steps of creating an (optimized) configuration `conf` as seen below. For more details on how to do this, check out the basic optimization example, which can be found [here](https://github.com/janattig/SudokuPlantDesign.jl/blob/main/examples/sudoku_basic_optimization.ipynb). With an optimized configuration `conf` at hand, one can proceed to create a field plan for the design. For such a field plan, additional information on the genotypes involved in the trial is added.

In the case of our example here, it is important that we distribute exactly 119 entries, which will be the length of our list later on. Then, optimization is carried out as in the other examples.

In [ ]:
conf = get_configuration([9,9],[2,2,2,2],3)

empty_plots!(conf, 1:2,3:4)
initialize_entries!(conf, 119)

cost_function(c) =  K_num_checks_equal_per_type(c) +
                    K_checks_per_type_per_block(c, 1)*20 +
                    K_neighbors_different_check_functional(c, d->0.5/(d^3)) +
                    K_neighbors_same_check_functional(c, d->1/(d^3))

updates = [UpdateNewCheckLabel(),UpdateSwapCheckCheck(),UpdateSwapCheckEntry()]

optimize_design!(
    conf,
    updates,
    cost_function,
    500000
);


show_configuration(conf, zoom=0.2, show_coordinates=true)

mkpath("output/")
savefig("output/field_plan_initial.pdf")

## 1 - Generating a labeled check configuration

With an optimized configuration `conf` at hand, one can proceed to create a field plan for the design. The first step in this endeavour is to create an upgraded version of the check configuration `conf`, a so-called *labeled check configuration* with the name `lconf`. This configuration not only contains the original check configuration but also labels and indices (position IDs) of each plant.

In [ ]:
lconf = LabeledCheckConfiguration(conf);

The labeled check configuration comes with its own `show_configuration` function with identical syntax to its check configuration counterpart. It not only plots the previously shown color pattern, but also the indices and additional labels on each plot.
```julia
show_configuration(
        lconf
        ;
        zoom = 1.0,
        title_zoom = 1.0,
        text_zoom = 1.0,
        cmap = "gist_rainbow",
        check_labels = true,
        dpi = 300,
        show_coordinates = false
    )
```

Note that upon initialization of a new labeled check configuration, all indices are set to `-1` and there are not labels, hence the rather empty figure.

In [ ]:
show_configuration(lconf, check_labels=true, show_coordinates=true, text_zoom=0.9)

## 2 Generating labels for the field plan

Let us add labels to the field plan. To do so, additional information on the genotypes involved in the trial is required. It is added as two *dataframes* with data for checks and entries respectively, which have to be of the following structure:
- first colum: genotype name
- further colums: additional information (optional, but have to be identical among the dataframes)

In the case of our example, the sheets `checks` and `entries` from the Excel file `input_augmented.xlsx` are read in and converted into the two dataframes. Each sheet contains the name of the entry in the first column, and in following columns further properties of the entries.

In [ ]:
entrydata = string.(DataFrame(XLSX.readtable("input_augmented.xlsx", "entries")));
replace!.(eachcol(entrydata), "missing" => "NA");

In [ ]:
checkdata = string.(DataFrame(XLSX.readtable("input_augmented.xlsx", "checks")));
replace!.(eachcol(checkdata), "missing" => "NA");

Of course, it is possible to create a dataframe without an Excel sheet. However in this case, it should be noted that the format of the dataframes has to be kept to the specifications of the cell above.

With the two dataframes `entrydata` and `checkdata` completed, one can now assign labels to the different plots. This is done with the function `fill_labels!`, which uses the dataframes to assign the values of the different columns as labels in the labeled check configuration:
```julia
fill_labels!(
    lconf,
    checkdata,
    entrydata
    ;
    shuffle_entries = true
)
```

The additional (optional) keyword argument `shuffle_indices` can be used to select if the (unreplicated) entries should be shuffled before being assigned to their respective fields.

Note, that the length of dataframes has to match the information of the check configuration, i.e. `checkdata` needs as many rows as check types and `entrydata` needs exactly as many rows as there are entries in the design.

In [ ]:
# fill the labels
fill_labels!(lconf, checkdata, entrydata)
# show the resulting configuration to see the labels
show_configuration(lconf, check_labels=true, show_coordinates=true, text_zoom=0.9)

## 3 Generating indices for the field plan

In the next step, the indices (position IDs) of the field plan are set. In total, there are several functions to assign indices to the different plots in the design:
```julia
fill_indices_snake_x!(lconf, di,dj)
fill_indices_snake_y!(lconf, di,dj)

fill_indices_lines_x!(lconf, di,dj)
fill_indices_lines_y!(lconf, di,dj)
```
All of these functions go through the different plots in the direction outlined by `di` and `dj` and assign labels either in a snake or line pattern (see examples belwo). 


Furthermore, optional arguments can be passed to all of these functions:
- `index_for_empty` if empty (missing) plots should get an index (position ID). Default: `false`
- The region in which the pattern should be applied (if not given by the boundaries of the design), given by:
    - `min_i` minimal x-position which is indexed
    - `max_i` maximal x-position which is indexed
    - `min_j` minimal y-position which is indexed
    - `max_j` maximal y-position which is indexed
- The explicit start position, given by xy-coordinates `i` and `j`
- `start_index` The index of the first plant (other indices are counted up from this). Default: `1`

In [ ]:
# fill indices by snake pattern along y axis, going in positive x direction (right) and positive y direction (up)
fill_indices_snake_y!(lconf, 1,1, index_for_empty=true)

show_configuration(lconf, check_labels=true, show_coordinates=true, text_zoom=0.9)

In [ ]:
# fill indices by lines pattern along x axis, going in negative x direction (left) and positive y direction (up)
fill_indices_lines_x!(lconf, -1,1, index_for_empty=true)

show_configuration(lconf, check_labels=true, show_coordinates=true, text_zoom=0.9)

In [ ]:
# fill indices by snake pattern along x axis which reflects at block boundaries
fill_indices_snake_x!(lconf, -1,1, index_for_empty=false, max_i=9)
fill_indices_snake_x!(lconf, 1,-1, index_for_empty=false, min_i=10, max_i=18, start_index=69)

show_configuration(lconf, check_labels=true, show_coordinates=true, text_zoom=0.9)

For exporting into a trial plan, the data of this optimized Sudoku-augmented design can now be converted back into a dataframe. This dataframe contains not only the genotype name and properties, but also their individual positions, xy-locations as well as information about their block. This dataframe can be further modified in julia before exporting it.

In [ ]:
df = get_dataframe(lconf)

As a first modification, all generic property columns are renamed to the the column names of the checkdata file.

In [ ]:
for (i,name) in enumerate(names(checkdata)[2:end])
    rename!(df,Symbol("property_"*string(i)) => Symbol(name))
end

df

As a second modification, additional columns are added.

In [ ]:
df[:, :year]       .= 2023
df[:, :extra_info] .= "myextrainfo"

df

Finally, the trial plan is created by writing the dataframe into an Excel file.

In [ ]:
mkpath("output/")
XLSX.writetable("output/augmented_design_final_design.xlsx", collect(eachcol(df)), names(df),overwrite=true)